<h1>Exercise 23</h1>
<p>Calculate 3 or more consecutive seats using</p>
<ul>
<li>Method 1: Lead lag</li>
<li>Method 2: Lead lag</li>
<li>Method 3: Lead lag</li>
</ul>
<strong>Expected outcomes:</strong>
<br>
<p>Method 1:</p>
<img src="Day-23-Expected_Output1.png" width="200">
<br>
<p>Method 2:</p>
<img src="Day-23-Expected_Output2.png" width="200">
<br>
<p>Method 3:</p>
<img src="Day-23-Expected_Output3.png" width="200">

In [1]:
import os
import sqlite3
import pandas as pd

db_name="exercise23.db"
conn=sqlite3.connect(db_name)
cursor=conn.cursor()

cursor.execute("""CREATE TABLE IF NOT EXISTS bms(seat_no INT,is_empty VARCHAR(10))""")
conn.commit()

bms_data=[
    (1,'N'),
    (2,'Y'),
    (3,'N'),
    (4,'Y'),
    (5,'Y'),
    (6,'Y'),
    (7,'N'),
    (8,'Y'),
    (9,'Y'),
    (10,'Y'),
    (11,'Y'),
    (12,'N'),
    (13,'Y'),
    (14,'Y')
]
conn.commit()

cursor.executemany("INSERT INTO bms VALUES(?,?)",bms_data)
conn.commit()

In [2]:
first_method="SELECT * FROM (SELECT * ,LAG(is_empty,1) OVER(ORDER BY seat_no) AS prev_1, LAG(is_empty,2) OVER(ORDER BY seat_no) AS prev_2, LEAD(is_empty,1) OVER(ORDER BY seat_no) AS next_1, LEAD(is_empty,2) OVER (ORDER BY seat_no) AS next_2 FROM bms) A WHERE is_empty='Y' AND prev_1='Y' and prev_2='Y' OR(is_empty='Y' and prev_1='Y' and next_1='Y') OR (is_empty='Y' and next_1='Y' and next_2='Y')"
df=pd.read_sql(first_method,conn)
print(df)

   seat_no is_empty prev_1 prev_2 next_1 next_2
0        4        Y      N      Y      Y      Y
1        5        Y      Y      N      Y      N
2        6        Y      Y      Y      N      Y
3        8        Y      N      Y      Y      Y
4        9        Y      Y      N      Y      Y
5       10        Y      Y      Y      Y      N
6       11        Y      Y      Y      N      Y


In [3]:
second_method="SELECT * FROM (SELECT *, SUM(CASE WHEN is_empty = 'Y' THEN 1 ELSE 0 END) OVER (ORDER BY seat_no ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS prev_2, SUM(CASE WHEN is_empty = 'Y' THEN 1 ELSE 0 END) OVER (ORDER BY seat_no ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) AS prev_next_1, SUM(CASE WHEN is_empty = 'Y' THEN 1 ELSE 0 END) OVER (ORDER BY seat_no ROWS BETWEEN CURRENT ROW AND 2 FOLLOWING) AS next_2 FROM BMS) A WHERE prev_2 = 3 OR prev_next_1 = 3 OR next_2 = 3"
df=pd.read_sql(second_method,conn)
print(df)

   seat_no is_empty  prev_2  prev_next_1  next_2
0        4        Y       2            2       3
1        5        Y       2            3       2
2        6        Y       3            2       2
3        8        Y       2            2       3
4        9        Y       2            3       3
5       10        Y       3            3       2
6       11        Y       3            2       2


In [4]:
third_method="WITH diff_num AS (SELECT *, ROW_NUMBER() OVER(ORDER BY seat_no) AS rn, seat_no - ROW_NUMBER() OVER(ORDER BY seat_no) AS diff FROM bms WHERE is_empty = 'Y'), cnt AS (SELECT diff, COUNT(1) AS c FROM diff_num GROUP BY diff HAVING COUNT(1) >= 3) SELECT * FROM diff_num WHERE diff IN (SELECT diff FROM cnt);"
df=pd.read_sql(third_method,conn)
print(df)
conn.close()

   seat_no is_empty  rn  diff
0        4        Y   2     2
1        5        Y   3     2
2        6        Y   4     2
3        8        Y   5     3
4        9        Y   6     3
5       10        Y   7     3
6       11        Y   8     3
